In [12]:
import cv2
import numpy as np
import os
import math
from heapq import nsmallest
import random
import functools

In [13]:
def contour_circularity(contour):
    # calculates how close is a contour to a circle shape
    perimeter = cv2.arcLength(contour,True)
    area = cv2.contourArea(contour)
    circularity = (4*math.pi*area)/(perimeter**2)
    return circularity

In [14]:
def contour_centroid(contour):
    # calculates the center point of a contour
    hull = cv2.convexHull(contour)
    M = cv2.moments(hull)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    return [cX, cY]

In [15]:
def get_hexagon(centroids, index):
    # finds the 6 closest centroids to a given centroid
    # returns the minimal ellipse which bounds the hexagon (which is made out of these centroids)
    x1, y1 = centroids[index]
    distances_dict = {}

    for j in range(len(centroids)):
        x2, y2 = centroids[j]
        if (x1 == x2) and (y1==y2):
            continue
        else:
            dist = math.hypot(x2-x1, y2-y1)
            distances_dict[(x2,y2)]=dist

    vertices_list = nsmallest(6, distances_dict, key = distances_dict.get)
    
    # the order of the vertices in 'vertices_list' is the order of connecting the dots of the hexagon
    # because of that, here we choose the order of vertices that creates a non-intersecting hexagon
    center = functools.reduce(lambda a, b: (a[0] + b[0], a[1] + b[1]), vertices_list, (0, 0))
    center = (center[0] / len(vertices_list), (center[1] / len(vertices_list)))
    vertices_list.sort(key = lambda a: math.atan2(a[1] - center[1], a[0] - center[0]))
    
    if len(vertices_list) == 6:
        return vertices_list
    else:
        return False


In [16]:
def is_edge_contour(contour):
    
    for dot in contour:
        width_val, height_val = dot[0][0], dot[0][1]
        # 684 and 1003 represent the last pixel of each image's height and width, respectively
        if width_val<=0 or (height_val<=0) or (width_val>=1003) or (height_val>=683):
            return True

    return False
    

In [17]:
def is_edge_point(point):
    
    width_val, height_val = point[0], point[1]
    # 684 and 1003 represent the last pixel of each image's height and width, respectively
    if width_val<=30 or (height_val<=50) or (width_val>=953) or (height_val>=633):
        return True

    return False
    

In [18]:
def pores_selector(img_dir):
    imageMat = cv2.imread(img_dir)
    
    #Select blue channel
    blueChannelMat = imageMat[:,:,0]
    
    # if an image is overly dark, CLAHE will be done on it
    if np.mean(blueChannelMat) <= 85:
        clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=[4,4])# 
        blueChannelMat = clahe.apply(blueChannelMat)
    
    #Mean filtering (the larger the filtering window, the more serious the image distortion is, the more blurred it is)
    blurMat = cv2.blur(blueChannelMat,(5,5))#5,5 means that the convolution kernel size is 5x5
    
    #Adaptive threshold segmentation
    thresMat = cv2.adaptiveThreshold(blurMat, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 15)

    #Find the contours of the image. The parameters indicate that only the outer contour is detected and 
    # all contour points are stored
    contours,hierarchy=cv2.findContours(thresMat,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    # mean pore size is calculated 
    allContoursArea = [cv2.contourArea(contours[i]) for i in range(len(contours))]
    mean_pore_size = np.mean(allContoursArea)
    # in this loop, we filter unwnated contours
    # and calculate centroid for each wanted contour
    centroids = []
        
    for i in range(len(contours)):
        
        #filters out pores which are on the edges of the image
        if is_edge_contour(contours[i]):
            continue
        
        contour_area = cv2.contourArea(contours[i])
         #filters contours which are too small to be a pore
        if contour_area < (mean_pore_size/2):
            continue

        else:
            # calculation of centroid
            cX, cY = contour_centroid(contours[i])
            centroids.append([cX,cY])
        

        # contours and centroids are drawn on the image
        cv2.drawContours(imageMat,[contours[i]],-1,(0,255,0),1)
        cv2.circle(imageMat, (cX, cY), 1, (255, 255, 255), 2)
        
          
    # this section is dedicated to recognition of hexagons
    # the list below stores the circularities of each hexagon
    hexagon_circ_list = []
    
    for i in range(len(centroids)):
        # filters polygons which their center is an edge point
        if is_edge_point(centroids[i]):
            continue
        
        # vertices_list - vertices of the hexagon
        vertices_list = get_hexagon(centroids, i)
        if vertices_list:
            hexagon_contour = np.array(vertices_list).reshape((-1,1,2)).astype(np.int32)
            # the closer 'hexagon_circ' is to 1 - the more it is similair to a circle
            hexagon_circ = contour_circularity(hexagon_contour)
            hexagon_circ_list.append(hexagon_circ)

        # hexagons are drawn on the image
        rand_num = random.randrange(0,len(centroids)-1)
        if i == rand_num:
            cv2.drawContours(imageMat,[hexagon_contour],-1,(255,0,0),5)
            cv2.circle(imageMat, centroids[i], 1, (0, 0, 150), 2)
        else:
            cv2.drawContours(imageMat,[hexagon_contour],-1,(255,0,0),1)
    
    
    # the closer of this value to 1 - the more symmetric the hexagonal grid is
    circularity_mean = np.mean(hexagon_circ_list)
  
    
    return imageMat, circularity_mean


In [21]:
# this block is used to show how 'pores_selector' works on a single image
img_dir = 'images\\hex\\images cropped\\S20072_P(VBCB-co-S)81P4VP19-159K-5s-200um-24wt%_003.tif'
#img_dir = 'images\\hex\\images cropped\\S17254_PS84P4VP16_210k_29%_60DMF-40THF_5s_003.tif'
#img_dir = 'images\\hex\\images cropped\\S16250_31%_PS83.2P4VP16.8_113k_10sAir_18sTHF_50DMF_50THF_001.tif'
im_with_selection, circularity_mean = pores_selector(img_dir)
cv2.imshow("b",im_with_selection)#Display image
print(circularity_mean)
cv2.waitKey()

0.6440637005444279


-1

In [ ]:
file_names = os.listdir('images\\hex\\images cropped')
count = 0
images_selected = {}
for file_name in file_names:
    img_dir = 'images\\hex\\images cropped\\'+file_name
    imageMat, circularity_mean = pores_selector(img_dir)
    images_selected[file_name] = circularity_mean

    count+=1 
    if count%100 == 0: # shows process
        print(count)